In [88]:

"""

啟用伺服器基本樣板

"""
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 
from linebot.exceptions import (
    InvalidSignatureError
)

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,ImageMessage, ImageSendMessage
)

# 載入設定檔

import json
secretFileContentJson=json.load(open("./line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/" )

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [89]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''
#收到圖片訊息,把圖片存到本地電腦
@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    message_content = line_bot_api.get_message_content(event.message.id)
    with open('./images/'+event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)
#把圖片修改,存回本地電腦
    import cv2
    import numpy as np
    from PIL import ImageFont, ImageDraw, Image

    path = './images/'+event.message.id+'.jpg'
    NewPath= './images/'+event.message.id+'Ver2.jpg'
    m1=cv2.imread(path, 1)

    m1 = Image.fromarray(m1)
    font = ImageFont.truetype("font/kaiu.ttf", int(170))
    ImageDraw.Draw(m1).text((3,3), ":D :D :D :D", (0, 0, 0), font)
    m1 = np.array(m1) 

    cv2.imwrite(NewPath, m1, [cv2.IMWRITE_JPEG_QUALITY, 100])
#把圖片跟消息做成List
    reply_message_list = [
    TextSendMessage(text="收到圖片。"),
    ImageSendMessage(original_content_url='https://2c3c4845.ngrok.io/images/'+ event.message.id +'Ver2.jpg' ,
    preview_image_url='https://2c3c4845.ngrok.io/images/'+ event.message.id +'Ver2.jpg'),
    ]
#發送List
    line_bot_api.reply_message(
    event.reply_token,
    reply_message_list
    )

In [ ]:
##小提醒,因為一個token只能用一次,所以要把對話存成一個list,然後用token去傳整個list
##小提醒,老師的Github有的點點有兩個 要記得改成一個點點

In [ ]:
'''

啟動Server

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Sep/2019 16:40:01] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 16:40:03] "GET /images/10515980381091Ver2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 16:42:17] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 16:42:18] "GET /images/10515991489517Ver2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 16:42:24] "GET /images/10515991489517Ver2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 16:42:36] "GET /images/10515991489517Ver2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [05/Sep/2019 16:42:36] "GET /images/10515980381091Ver2.jpg HTTP/1.1" 200 -
